In [1]:
import dill
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from collections import defaultdict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import jaccard_score
import os
import argparse
from util import Metrics
import time

import sys
sys.path.append('..')
from util import multi_label_metric
model_name = 'LR'
resume_path = 'saved/{}/Epoch_49_JA_0.4603_DDI_0.07427.model'.format(model_name)

parser = argparse.ArgumentParser()
parser.add_argument('--Test', action='store_true', default=False, help="test mode")
parser.add_argument('--FT', action='store_true', default=False, help="Fine Tune")
parser.add_argument('--datadir', type=str, default="../data/", help='dimension')
parser.add_argument('--ftfile', type=str, default="emm", help='finetune file')
parser.add_argument('--cuda', type=int, default=-1, help='use cuda')
parser.add_argument('--epoch', type=int, default=400, help='# of epoches')
parser.add_argument('--early_stop', type=int, default=30, help='early stop number')
parser.add_argument('--resume_path', type=str, default=resume_path, help='resume path')
parser.add_argument('--lr', type=float, default=5e-4, help='learning rate')
parser.add_argument('--model_name', type=str, default=model_name, help="model name")
parser.add_argument('--seed', type=int, default=1029, help='use cuda')

args = parser.parse_args()
args.MIMIC=3
if not os.path.exists(os.path.join("saved", args.model_name)):
    os.makedirs(os.path.join("saved", args.model_name))
def create_dataset(data, diag_voc, pro_voc, med_voc):
    i1_len = len(diag_voc.idx2word)
    i2_len = len(pro_voc.idx2word)
    output_len = len(med_voc.idx2word)
    input_len = i1_len + i2_len
    X = []
    y = []
    for patient in data:
        for visit in patient:
            i1 = visit[0]
            i2 = visit[1]
            o = visit[2]

            multi_hot_input = np.zeros(input_len)
            multi_hot_input[i1] = 1
            multi_hot_input[np.array(i2) + i1_len] = 1

            multi_hot_output = np.zeros(output_len)
            multi_hot_output[o] = 1

            X.append(multi_hot_input)
            y.append(multi_hot_output)

    return np.array(X), np.array(y)

In [2]:
# grid_search = False
# data_path = os.path.join(args.datadir, 'records_final_4.pkl')
# voc_path = os.path.join(args.datadir, 'voc_final_4.pkl')
data_path = os.path.join(args.datadir, 'records_final.pkl')
voc_path = os.path.join(args.datadir, 'voc_final.pkl')

data = dill.load(open(data_path, 'rb'))
voc = dill.load(open(voc_path, 'rb'))
diag_voc, pro_voc, med_voc = voc['diag_voc'], voc['pro_voc'], voc['med_voc']
metric_obj = Metrics(data, med_voc, args)

thre idx: 57


In [3]:
for epoch in range(1):

    np.random.seed(args.seed)
    np.random.shuffle(data)
    split_point = int(len(data) * 2 / 3)
    data_train = data[:split_point]
    eval_len = int(len(data[split_point:]) / 2)
    data_eval = data[split_point+eval_len:]
    data_test = data[split_point:split_point + eval_len]
    print('1')
    train_X, train_y = create_dataset(data_train, diag_voc, pro_voc, med_voc)
    test_X, test_y = create_dataset(data_test, diag_voc, pro_voc, med_voc)
    eval_X, eval_y = create_dataset(data_eval, diag_voc, pro_voc, med_voc)
    model = LogisticRegression()
    classifier = OneVsRestClassifier(model)
    print('2')

    tic = time.time()
    classifier.fit(train_X, train_y)
    print('3')

    fittime = time.time() - tic
    print ('fitting time: {}'.format(fittime))


    result = []
    for _ in range(1):
        # index = np.random.choice(np.arange(len(test_X)), round(len(test_X) * 0.8), replace=True)
        test_sample = test_X  # [index]
        y_sample = test_y  # [index]
        y_pred = classifier.predict(test_sample)
        pretime = time.time() - tic
        print ('inference time: {}'.format(pretime))

        y_prob = classifier.predict_proba(test_sample)

        metric_obj.set_data(y_sample, y_pred, y_prob, save=args.Test)
        ja, prauc, avg_p, avg_r, avg_f1 = metric_obj.run()
        # ja, prauc, avg_p, avg_r, avg_f1 = multi_label_metric(y_sample, y_pred, y_prob)

        # ddi rate
        ddi_adj_path = os.path.join(args.datadir, 'ddi_A_final.pkl')
        # ddi_adj_path = os.path.join(args.datadir, 'ddi_A_final_4.pkl')
        ddi_A = dill.load(open(ddi_adj_path, 'rb'))
        all_cnt = 0
        dd_cnt = 0
        med_cnt = 0
        visit_cnt = 0
        for adm in y_pred:
            med_code_set = np.where(adm==1)[0]
            visit_cnt += 1
            med_cnt += len(med_code_set)
            for i, med_i in enumerate(med_code_set):
                for j, med_j in enumerate(med_code_set):
                    if j <= i:
                        continue
                    all_cnt += 1
                    if ddi_A[med_i, med_j] == 1 or ddi_A[med_j, med_i] == 1:
                        dd_cnt += 1
        ddi_rate = dd_cnt / all_cnt
        result.append([ddi_rate, ja, avg_f1, prauc, med_cnt / visit_cnt])
    
    result = np.array(result)
    mean = result.mean(axis=0)
    std = result.std(axis=0)

    outstring = ""
    for m, s in zip(mean, std):
        outstring += "{:.4f} $\pm$ {:.4f} & ".format(m, s)

    print (outstring)

    tic = time.time()
    print('Epoch: {}, DDI Rate: {:.4}, Jaccard: {:.4}, PRAUC: {:.4}, AVG_PRC: {:.4}, AVG_RECALL: {:.4}, AVG_F1: {:.4}, AVG_MED: {:.4}\n'.format(
        epoch, ddi_rate, ja, prauc, avg_p, avg_r, avg_f1, med_cnt / visit_cnt
        ))

    history = defaultdict(list)
    history['fittime'].append(fittime)
    history['pretime'].append(pretime)
    history['jaccard'].append(ja)
    history['ddi_rate'].append(ddi_rate)
    history['avg_p'].append(avg_p)
    history['avg_r'].append(avg_r)
    history['avg_f1'].append(avg_f1)
    history['prauc'].append(prauc)

dill.dump(history, open(os.path.join('saved', model_name, 'history.pkl'), 'wb'))
print('Avg_Fittime: {:.8}, Avg_Pretime: {:.8}, Avg_Jaccard: {:.4}, Avg_DDI: {:.4}, Avg_p: {:.4}, Avg_r: {:.4}, \
        Avg_f1: {:.4}, AVG_PRC: {:.4}\n'.format(
    np.mean(history['fittime']),
    np.mean(history['pretime']),
    np.mean(history['jaccard']),
    np.mean(history['ddi_rate']),
    np.mean(history['avg_p']),
    np.mean(history['avg_r']),
    np.mean(history['avg_f1']),
    np.mean(history['prauc'])
    ))

1
2
3
fitting time: 128.0398235321045
inference time: 129.1986653804779
adverse DDI number: -1.0000, Jaccard: 0.4936,  PRAUC: 0.7578, AVG_PRC: 0.7283, AVG_RECALL: 0.6109, AVG_F1: 0.6503, AVG_MED: 16.1644

0.0838 $\pm$ 0.0000 & 0.4936 $\pm$ 0.0000 & 0.6503 $\pm$ 0.0000 & 0.7578 $\pm$ 0.0000 & 16.1644 $\pm$ 0.0000 & 
Epoch: 0, DDI Rate: 0.08378, Jaccard: 0.4936, PRAUC: 0.7578, AVG_PRC: 0.7283, AVG_RECALL: 0.6109, AVG_F1: 0.6503, AVG_MED: 16.16

Avg_Fittime: 128.03982, Avg_Pretime: 129.19867, Avg_Jaccard: 0.4936, Avg_DDI: 0.08378, Avg_p: 0.7283, Avg_r: 0.6109,         Avg_f1: 0.6503, AVG_PRC: 0.7578



In [4]:
result

array([[ 0.08378316,  0.49357555,  0.65027099,  0.75781962, 16.16435644]])

In [5]:
y_sample

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [6]:
ja

0.4935755455881565

In [7]:
import utils_

ModuleNotFoundError: No module named 'utils_'

In [8]:
sys.path.append('../..')
import utils_

ModuleNotFoundError: No module named 'utils_'

In [9]:
def multi_label_metric2(y_gt, y_pred, y_prob):
    # Jaccard系数
    def jaccard(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            union = set(out_list) | set(target)
            jaccard_score = 0 if len(union)==0 else len(inter) / len(union)
            score.append(jaccard_score)
        return np.mean(score)
    # 平均精确率
    def average_prc(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            prc_score = 0 if len(out_list) == 0 else len(inter) / len(out_list)
            score.append(prc_score)
        return score
    # 平均召回率
    def average_recall(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            recall_score = 0 if len(target) == 0 else len(inter) / len(target)
            score.append(recall_score)
        return score
    # 平均F1
    def average_f1(average_prc, average_recall):
        score = []
        for idx in range(len(average_prc)):
            if average_prc[idx] + average_recall[idx] == 0:
                score.append(0)
            else:
                score.append(2*average_prc[idx]*average_recall[idx] / (average_prc[idx] + average_recall[idx]))
        return score
    # Macro F1
    def f1(y_gt, y_pred):
        all_micro = []
        for b in range(y_gt.shape[0]):
            all_micro.append(f1_score(y_gt[b], y_pred[b], average='macro'))
        return np.mean(all_micro)
    # 平均精确率
    def precision_auc(y_gt, y_prob):
        all_micro = []
        for b in range(len(y_gt)):
            all_micro.append(average_precision_score(y_gt[b], y_prob[b], average='macro'))
        return np.mean(all_micro)
    # Macro F1
    f1 = f1(y_gt, y_pred)
    # 平均精确率
    prauc = precision_auc(y_gt, y_prob)
    # Jaccard系数
    ja = jaccard(y_gt, y_pred)
    # Precision, Recall, F1
    avg_prc = average_prc(y_gt, y_pred)
    avg_recall = average_recall(y_gt, y_pred)
    avg_f1 = average_f1(avg_prc, avg_recall)
    return ja, prauc, np.mean(avg_prc), np.mean(avg_recall), np.mean(avg_f1)

In [10]:
multi_label_metric2(test_y, y_pred, y_prob)

NameError: name 'f1_score' is not defined

In [11]:
from sklearn.metrics import f1_score, average_precision_score;
def multi_label_metric2(y_gt, y_pred, y_prob):
    # Jaccard系数
    def jaccard(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            union = set(out_list) | set(target)
            jaccard_score = 0 if len(union)==0 else len(inter) / len(union)
            score.append(jaccard_score)
        return np.mean(score)
    # 平均精确率
    def average_prc(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            prc_score = 0 if len(out_list) == 0 else len(inter) / len(out_list)
            score.append(prc_score)
        return score
    # 平均召回率
    def average_recall(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            recall_score = 0 if len(target) == 0 else len(inter) / len(target)
            score.append(recall_score)
        return score
    # 平均F1
    def average_f1(average_prc, average_recall):
        score = []
        for idx in range(len(average_prc)):
            if average_prc[idx] + average_recall[idx] == 0:
                score.append(0)
            else:
                score.append(2*average_prc[idx]*average_recall[idx] / (average_prc[idx] + average_recall[idx]))
        return score
    # Macro F1
    def f1(y_gt, y_pred):
        all_micro = []
        for b in range(y_gt.shape[0]):
            all_micro.append(f1_score(y_gt[b], y_pred[b], average='macro'))
        return np.mean(all_micro)
    # 平均精确率
    def precision_auc(y_gt, y_prob):
        all_micro = []
        for b in range(len(y_gt)):
            all_micro.append(average_precision_score(y_gt[b], y_prob[b], average='macro'))
        return np.mean(all_micro)
    # Macro F1
    f1 = f1(y_gt, y_pred)
    # 平均精确率
    prauc = precision_auc(y_gt, y_prob)
    # Jaccard系数
    ja = jaccard(y_gt, y_pred)
    # Precision, Recall, F1
    avg_prc = average_prc(y_gt, y_pred)
    avg_recall = average_recall(y_gt, y_pred)
    avg_f1 = average_f1(avg_prc, avg_recall)
    return ja, prauc, np.mean(avg_prc), np.mean(avg_recall), np.mean(avg_f1)

In [12]:
multi_label_metric2(test_y, y_pred, y_prob)

(0.4935755455881565,
 0.7578196215931153,
 0.7282814174760223,
 0.6108980584707616,
 0.6502709857843286)

In [13]:
ja

0.4935755455881565

In [14]:
prauc

0.7578196215931153

In [15]:
avg_p

0.7282814174760223

In [16]:
avg_f1

0.6502709857843286